![flow_diagram](https://drive.google.com/uc?export=view&id=1mIm6g1LXoH6c4YSI84xqHlk8QTia5KMS)


This collab presents a demo of code-switch detection using trained BiGRU-with-attn with pre-trained non-contextual sub-word embeddings using Skipgram model with 300 dimensions. The RNN model is trained and validated on the Hansard training and validation set. 

Flow diagram: STEP 1 is done, we are looking at STEP 2 here.

In [ ]:
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.metrics import f1_score
import string

from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    classification_report
)


In [ ]:
#!pip install numpy==1.19.5

In [ ]:
## loading trained model. A summary of the model architecture is also presented.
loaded_model = tf.keras.models.load_model('bilstm-Maori-Eng-300SG.h5')



In [3]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 300)          25531200  
_________________________________________________________________
dropout (Dropout)            (None, 250, 300)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dense (Dense)                (None, 3)                 387       
Total params: 25,718,467
Trainable params: 187,267
Non-trainable params: 25,531,200
_________________________________________________________________


In [4]:
## loading tokenizer. 
with open('tokenizerbilstm-Maori-Eng-300SG.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)


In [5]:
df = pd.read_csv("rmt_sample.csv")
df = df.drop(['url', 'user.id', 'date', 'content_with_emojis', 'total_words', 'percent_maori',
       'conversation_id', 'in_reply_to_user_id', 'lang', 'source_label',
       'error', 'favourites', 'like_count', 'quote_count', 'retweet_count',
       'reply_count', 'year', 'media', 'outlinks'],axis = 1)
df.columns = ['id','text','maori_words','number_of_words']
df.head()

,id,text,maori_words,number_of_words
0,77657422',any tuhoe kei roto i te whare?,"['tuhoe', 'kei', 'roto', 'te', 'whare']",5
1,553111032',Haere mai ki te L:Waitahananui,"['haere', 'mai', 'ki', 'te']",4
2,585594182',Kua hinga te totara nui o te wao tapu. Moe mai...,"['kua', 'hinga', 'te', 'totara', 'nui', 'o', '...",12
3,829531914',"He nui tangata e heke ana ki te Pō, he iti tan...","['he', 'nui', 'tangata', 'e', 'heke', 'ana', '...",18
4,863218149',"<user> oma ika, oma ika, oma oma oma","['oma', 'ika', 'oma', 'ika', 'oma', 'oma', 'oma']",7


In [6]:
def remove_punc(string):
    punc = '''!()-[]{};:'" <>./?@#$%^&*_~'''
    for ele in string:  
        if ele in punc:  
            string = string.replace(ele, "") 
    return string
 
wb = []
idx = []


for ind, row in df.iterrows():
    x = row['text']
    idf = row['id']
    trial = pd.DataFrame()
    y1 = x.split()
    y = [remove_punc(i) for i in y1] 
    for i in y:
        seq1= tokenizer.texts_to_sequences([i])
        padded1 = pad_sequences(seq1, maxlen=250)
        predict1=loaded_model.predict(padded1) 
        classw=np.argmax(predict1,axis=1)
        if int(classw[0]) == 1:
            wb.append(i)
            idx.append(idf)
        else:
            continue
    trial['words_pred'] = wb
    trial['id'] = idx

pred = trial.groupby('id', as_index=False).agg(lambda x: x.tolist())

result = pd.merge(df, pred, on="id", how="inner")
total_number_maori_words = result['number_of_words'].sum()


r = result.drop(['text'],axis=1)

r['unique_maori_words'] = r['maori_words'].str.lower().str.split(',').apply(set).sort_values()

r['unique_pred_words'] = r['words_pred'].astype(str).str.lower().str.split(',').apply(set).sort_values()


2022-01-16 20:16:09.135781: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-16 20:16:09.157781: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz
2022-01-16 20:16:09.579408: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-16 20:16:09.861679: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-16 20:16:10.603781: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-01-16 20:16:10.604060: W tensorflow/core/framework/op_kernel.cc:1763] OP_REQUIRES failed at cudnn_rnn_ops.cc:1514 : Unknown: Fail to find the dnn implementation.
2022-01-16 20:16:10.623532: E tensorflow/stream_executor/cuda/cuda_dnn.cc:336] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/bidirectional/forward_lstm/PartitionedCall]] [Op:__inference_predict_function_2156]

Function call stack:
predict_function -> predict_function -> predict_function


In [ ]:
wb1 = []
idx1 = []

count = 0

for ind, row in r.iterrows():
    count = count + 1
    mw = row['unique_maori_words']
    pw = row['unique_pred_words']
    #print(count)
    first_list = [remove_punc(i) for i in mw] 
    sec_list = [remove_punc(i) for i in pw]
    idf = row['id']
    diff = list(set(first_list) - set(sec_list))
    t = pd.DataFrame()
    wb1.append(diff)
    idx1.append(idf)
t['difference'] = wb1
t['id'] = idx1

t['length'] = t['difference'].str.len()

t = t[t.length > 0]

total_diff = t['length'].sum()

print(total_number_maori_words)
print(total_diff)